# Cifar10 Classification with Keras

Keras exists to make coding deep neural networks simpler. To demonstrate just how easy it is, you’re going to use Keras to build a convolutional neural network in a few dozen lines of code.

You’ll be connecting the concepts from the previous lessons to the methods that Keras provides.

## Overview

Here are the steps you'll take to build the network:

1. Load the training data.
2. Preprocess the data.
3. Build a feedforward neural network to classify traffic signs.
4. Build a convolutional neural network to classify traffic signs.
5. Evaluate the final neural network on testing data.



In [1]:
import pickle
import numpy as np
import math

# Fix error with TF and Keras
import tensorflow as tf
tf.python.control_flow_ops = tf

print('Modules loaded.')

Modules loaded.


## Load the Data

Start by importing the data from the pickle file.

In [22]:
#load cifar10 to check the performance
from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

## Preprocess the Data

1. Shuffle the data
2. Normalize the features using Min-Max scaling between -0.5 and 0.5
3. One-Hot Encode the labels

### Shuffle the data
Hint: You can use the [scikit-learn shuffle](http://scikit-learn.org/stable/modules/generated/sklearn.utils.shuffle.html) function to shuffle the data.

In [4]:
# TODO: Shuffle the data
from sklearn.utils import shuffle

X_train, y_train = shuffle(X_train, y_train)


### Normalize the features
Hint: You solved this in [TensorFlow lab](https://github.com/udacity/CarND-TensorFlow-Lab/blob/master/lab.ipynb) Problem 1.

In [5]:
# TODO: Normalize the data features to the variable X_normalized

def normalize_minmax_scale(image_data,feature_range):
    """
    Normalize the image data with Min-Max scaling
    :param image_data: The image data to be normalized
    :param range: range of output features
    :return: Normalized image data
    """
    a, b = feature_range
    RGB_min = 0
    RGB_max = 255
    return a + ( ( (image_data - RGB_min)*(b - a) )/( RGB_max - RGB_min ) )

min_max_range = (-0.5,0.5)
X_normalized = normalize_minmax_scale(X_train,min_max_range)



In [6]:
# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert math.isclose(np.min(X_normalized), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_normalized), 0.5, abs_tol=1e-5), 'The range of the training data is: {} to {}.  It must be -0.5 to 0.5'.format(np.min(X_normalized), np.max(X_normalized))
print('Tests passed.')

Tests passed.


### One-Hot Encode the labels
Hint: You can use the [scikit-learn LabelBinarizer](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelBinarizer.html) function to one-hot encode the labels.

In [8]:
# TODO: One Hot encode the labels to the variable y_one_hot
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
y_one_hot = lb.fit_transform(y_train)


## Keras Sequential Model
```python
from keras.models import Sequential

# Create the Sequential model
model = Sequential()
```
The `keras.models.Sequential` class is a wrapper for the neural network model. Just like many of the class models in scikit-learn, it provides common functions like `fit()`, `evaluate()`, and `compile()`.  We'll cover these functions as we get to them.  Let's start looking at the layers of the model.

## Keras Layer
A Keras layer is just like a neural network layer.  It can be fully connected, max pool, activation, etc.  You can add a layer to the model using the model's `add()` function.  For example, a simple model would look like this:
```python
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten

# Create the Sequential model
model = Sequential()

# 1st Layer - Add a flatten layer
model.add(Flatten(input_shape=(32, 32, 3)))

# 2nd Layer - Add a fully connected layer
model.add(Dense(100))

# 3rd Layer - Add a ReLU activation layer
model.add(Activation('relu'))

# 4th Layer - Add a fully connected layer
model.add(Dense(60))

# 5th Layer - Add a ReLU activation layer
model.add(Activation('relu'))
```
Keras will automatically infer the shape of all layers after the first layer.  This means you only have to set the input dimensions for the first layer.

The first layer from above, `model.add(Flatten(input_shape=(32, 32, 3)))`, sets the input dimension to (32, 32, 3) and output dimension to (3072=32\*32\*3).  The second layer takes in the output of the first layer and sets the output dimenions to (100).  This chain of passing output to the next layer continues until the last layer, which is the output of the model.

## Build a Multi-Layer Feedforward Network

Build a multi-layer feedforward neural network to classify the traffic sign images.

1. Set the first layer to a `Flatten` layer with the `input_shape` set to (32, 32, 3)
2. Set the second layer to `Dense` layer width to 128 output. 
3. Use a ReLU activation function after the second layer.
4. Set the output layer width to 10, since there are 10 classes in the dataset.
5. Use a softmax activation function after the output layer.

To get started, review the Keras documentation about [models](https://keras.io/models/sequential/) and [layers](https://keras.io/layers/core/).

The Keras example of a [Multi-Layer Perceptron](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py) network is similar to what you need to do here. Use that as a guide, but keep in mind that there are a number of differences.

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

model = Sequential()
# TODO: Build a Multi-layer feedforward neural network with Keras here.
# 1st Layer - Add a flatten layer
model.add(Flatten(input_shape=(32, 32, 3)))
# 2nd Layer - Add a fully connected layer
model.add(Dense(128))
# 3rd Layer - Add a ReLU activation layer
model.add(Activation('relu'))
# 4th Layer - Add a fully connected layer
model.add(Dense(10))
# 5th Layer - Add a softmax activation layer
model.add(Activation('softmax'))

In [10]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
from keras.layers.core import Dense, Activation, Flatten
from keras.activations import relu, softmax

def check_layers(layers, true_layers):
    assert len(true_layers) != 0, 'No layers found'
    for layer_i in range(len(layers)):
        assert isinstance(true_layers[layer_i], layers[layer_i]), 'Layer {} is not a {} layer'.format(layer_i+1, layers[layer_i].__name__)
    assert len(true_layers) == len(layers), '{} layers found, should be {} layers'.format(len(true_layers), len(layers))

check_layers([Flatten, Dense, Activation, Dense, Activation], model.layers)

assert model.layers[0].input_shape == (None, 32, 32, 3), 'First layer input shape is wrong, it should be (32, 32, 3)'
assert model.layers[1].output_shape == (None, 128), 'Second layer output is wrong, it should be (128)'
assert model.layers[2].activation == relu, 'Third layer not a relu activation layer'
assert model.layers[3].output_shape == (None, 10), 'Fourth layer output is wrong, it should be (43)'
assert model.layers[4].activation == softmax, 'Fifth layer not a softmax activation layer'
print('Tests passed.')

Tests passed.


## Training a Sequential Model
You built a multi-layer neural network in Keras, now let's look at training a neural network.
```python
from keras.models import Sequential
from keras.layers.core import Dense, Activation

model = Sequential()
...

# Configures the learning process and metrics
model.compile('sgd', 'mean_squared_error', ['accuracy'])

# Train the model
# History is a record of training loss and metrics
history = model.fit(x_train_data, Y_train_data, batch_size=128, nb_epoch=2, validation_split=0.2)

# Calculate test score
test_score = model.evaluate(x_test_data, Y_test_data)
```
The code above configures, trains, and tests the model.  The line `model.compile('sgd', 'mean_squared_error', ['accuracy'])` configures the model's optimizer to `'sgd'`(stochastic gradient descent), the loss to `'mean_squared_error'`, and the metric to `'accuracy'`.  

You can find more optimizers [here](https://keras.io/optimizers/), loss functions [here](https://keras.io/objectives/#available-objectives), and more metrics [here](https://keras.io/metrics/#available-metrics).

To train the model, use the `fit()` function as shown in `model.fit(x_train_data, Y_train_data, batch_size=128, nb_epoch=2, validation_split=0.2)`.  The `validation_split` parameter will split a percentage of the training dataset to be used to validate the model.  The model can be further tested with the test dataset using the `evaluation()` function as shown in the last line.

## Train the Network

1. Compile the network using adam optimizer and categorical_crossentropy loss function.
2. Train the network for ten epochs and validate with 20% of the training data.

In [11]:
# TODO: Compile and train the model here.
# Configures the learning process and metrics
model.compile('Adam', 'categorical_crossentropy', ['accuracy'])
# History is a record of training loss and metrics
history = model.fit(X_normalized, y_one_hot, batch_size=128, nb_epoch=10, validation_split=0.2)

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 66s - loss: 1.6786 - acc: 0.4124 - val_loss: 1.5862 - val_acc: 0.4415
Epoch 2/10
40000/40000 [==============================] - 1s - loss: 1.4741 - acc: 0.4850 - val_loss: 1.5184 - val_acc: 0.4680
Epoch 3/10
40000/40000 [==============================] - 1s - loss: 1.3753 - acc: 0.5227 - val_loss: 1.4855 - val_acc: 0.4840
Epoch 4/10
40000/40000 [==============================] - 1s - loss: 1.3116 - acc: 0.5458 - val_loss: 1.4589 - val_acc: 0.4955
Epoch 5/10
40000/40000 [==============================] - 1s - loss: 1.2609 - acc: 0.5639 - val_loss: 1.4601 - val_acc: 0.4986
Epoch 6/10
40000/40000 [==============================] - 1s - loss: 1.2081 - acc: 0.5809 - val_loss: 1.4716 - val_acc: 0.4988
Epoch 7/10
40000/40000 [==============================] - 1s - loss: 1.1677 - acc: 0.5952 - val_loss: 1.4585 - val_acc: 0.5072
Epoch 8/10
40000/40000 [==============================] - 1s

## Convolutions
1. Re-construct the previous network
2. Add a [convolutional layer](https://keras.io/layers/convolutional/#convolution2d) with 32 filters, a 3x3 kernel, and valid padding before the flatten layer.
3. Add a ReLU activation after the convolutional layer.

Hint 1: The Keras example of a [convolutional neural network](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py) for MNIST would be a good example to review.

In [12]:
# TODO: Re-construct the network and add a convolutional layer before the flatten layer.
from keras.layers import Convolution2D, MaxPooling2D

model = Sequential()

model.add(Convolution2D(32, 3, 3, border_mode='valid', input_shape=(32, 32,3)))

model.add(Activation('relu'))

model.add(Flatten())

model.add(Dense(128))

model.add(Activation('relu'))

model.add(Dense(10))

model.add(Activation('softmax'))

In [14]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
from keras.layers.core import Dense, Activation, Flatten
from keras.layers.convolutional import Convolution2D

check_layers([Convolution2D, Activation, Flatten, Dense, Activation, Dense, Activation], model.layers)

assert model.layers[0].input_shape == (None, 32, 32, 3), 'First layer input shape is wrong, it should be (32, 32, 3)'
assert model.layers[0].nb_filter == 32, 'Wrong number of filters, it should be 32'
assert model.layers[0].nb_col == model.layers[0].nb_row == 3, 'Kernel size is wrong, it should be a 3x3'
assert model.layers[0].border_mode == 'valid', 'Wrong padding, it should be valid'

model.compile('adam', 'categorical_crossentropy', ['accuracy'])
history = model.fit(X_normalized, y_one_hot, batch_size=128, nb_epoch=10, validation_split=0.2)
print('Tests passed.')

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 2s - loss: 0.9342 - acc: 0.6742 - val_loss: 1.0987 - val_acc: 0.6141
Epoch 2/10
40000/40000 [==============================] - 2s - loss: 0.7765 - acc: 0.7324 - val_loss: 1.1462 - val_acc: 0.6045
Epoch 3/10
40000/40000 [==============================] - 2s - loss: 0.6501 - acc: 0.7771 - val_loss: 1.1238 - val_acc: 0.6262
Epoch 4/10
40000/40000 [==============================] - 2s - loss: 0.5155 - acc: 0.8256 - val_loss: 1.2052 - val_acc: 0.6195
Epoch 5/10
40000/40000 [==============================] - 2s - loss: 0.4003 - acc: 0.8691 - val_loss: 1.2747 - val_acc: 0.6207
Epoch 6/10
40000/40000 [==============================] - 2s - loss: 0.2983 - acc: 0.9069 - val_loss: 1.3584 - val_acc: 0.6225
Epoch 7/10
40000/40000 [==============================] - 2s - loss: 0.2136 - acc: 0.9372 - val_loss: 1.4841 - val_acc: 0.6254
Epoch 8/10
40000/40000 [==============================] - 2s 

## Pooling
1. Re-construct the network
2. Add a 2x2 [max pooling layer](https://keras.io/layers/pooling/#maxpooling2d) immediately following your convolutional layer.

In [15]:
# TODO: Re-construct the network and add a pooling layer after the convolutional layer.
model = Sequential()

model.add(Convolution2D(32, 3, 3, border_mode='valid', input_shape=(32, 32,3)))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Activation('relu'))

model.add(Flatten())

model.add(Dense(128))

model.add(Activation('relu'))

model.add(Dense(10))

model.add(Activation('softmax'))

In [17]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
from keras.layers.core import Dense, Activation, Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

check_layers([Convolution2D, MaxPooling2D, Activation, Flatten, Dense, Activation, Dense, Activation], model.layers)
assert model.layers[1].pool_size == (2, 2), 'Second layer must be a max pool layer with pool size of 2x2'

model.compile('adam', 'categorical_crossentropy', ['accuracy'])
history = model.fit(X_normalized, y_one_hot, batch_size=128, nb_epoch=10, validation_split=0.2)
print('Tests passed.')

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 2s - loss: 1.0402 - acc: 0.6376 - val_loss: 1.0935 - val_acc: 0.6167
Epoch 2/10
40000/40000 [==============================] - 2s - loss: 0.9295 - acc: 0.6764 - val_loss: 1.0604 - val_acc: 0.6292
Epoch 3/10
40000/40000 [==============================] - 2s - loss: 0.8404 - acc: 0.7084 - val_loss: 1.0391 - val_acc: 0.6400
Epoch 4/10
40000/40000 [==============================] - 2s - loss: 0.7581 - acc: 0.7367 - val_loss: 1.0268 - val_acc: 0.6466
Epoch 5/10
40000/40000 [==============================] - 2s - loss: 0.6809 - acc: 0.7655 - val_loss: 1.0138 - val_acc: 0.6598
Epoch 6/10
40000/40000 [==============================] - 2s - loss: 0.6062 - acc: 0.7917 - val_loss: 1.0236 - val_acc: 0.6574
Epoch 7/10
40000/40000 [==============================] - 2s - loss: 0.5322 - acc: 0.8185 - val_loss: 1.0318 - val_acc: 0.6602
Epoch 8/10
40000/40000 [==============================] - 2s 

## Dropout
1. Re-construct the network
2. Add a [dropout](https://keras.io/layers/core/#dropout) layer after the pooling layer. Set the dropout rate to 50%.

In [18]:
# TODO: Re-construct the network and add dropout after the pooling layer.
from keras.layers import Dropout

model = Sequential()

model.add(Convolution2D(32, 3, 3, border_mode='valid', input_shape=(32, 32,3)))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.5))

model.add(Activation('relu'))

model.add(Flatten())

model.add(Dense(128))

model.add(Activation('relu'))

model.add(Dense(10))

model.add(Activation('softmax'))

In [19]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

check_layers([Convolution2D, MaxPooling2D, Dropout, Activation, Flatten, Dense, Activation, Dense, Activation], model.layers)
assert model.layers[2].p == 0.5, 'Third layer should be a Dropout of 50%'

model.compile('adam', 'categorical_crossentropy', ['accuracy'])
history = model.fit(X_normalized, y_one_hot, batch_size=128, nb_epoch=10, validation_split=0.2)
print('Tests passed.')

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 2s - loss: 1.5210 - acc: 0.4635 - val_loss: 1.3193 - val_acc: 0.5389
Epoch 2/10
40000/40000 [==============================] - 2s - loss: 1.2452 - acc: 0.5638 - val_loss: 1.1860 - val_acc: 0.5858
Epoch 3/10
40000/40000 [==============================] - 2s - loss: 1.1391 - acc: 0.5995 - val_loss: 1.1227 - val_acc: 0.6089
Epoch 4/10
40000/40000 [==============================] - 2s - loss: 1.0683 - acc: 0.6266 - val_loss: 1.0836 - val_acc: 0.6260
Epoch 5/10
40000/40000 [==============================] - 2s - loss: 1.0015 - acc: 0.6496 - val_loss: 1.0491 - val_acc: 0.6376
Epoch 6/10
40000/40000 [==============================] - 2s - loss: 0.9464 - acc: 0.6687 - val_loss: 0.9982 - val_acc: 0.6559
Epoch 7/10
40000/40000 [==============================] - 2s - loss: 0.8941 - acc: 0.6857 - val_loss: 0.9726 - val_acc: 0.6606
Epoch 8/10
40000/40000 [==============================] - 2s 

## Optimization
Congratulations! You've built a neural network with convolutions, pooling, dropout, and fully-connected layers, all in just a few lines of code.

Have fun with the model and see how well you can do! Add more layers, or regularization, or different padding, or batches, or more training epochs.

What is the best validation accuracy you can achieve?

In [20]:
# TODO: Build a model
model = Sequential()

model.add(Convolution2D(100, 3, 3, border_mode='valid', input_shape=(32, 32,3)))

model.add(Activation('tanh'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(150, 4, 4, border_mode='valid'))

model.add(Activation('tanh'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(250, 3, 3, border_mode='valid'))

model.add(Activation('tanh'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(300))

model.add(Activation('tanh'))

model.add(Dropout(0.5))

model.add(Dense(10))

model.add(Activation('softmax'))
# TODO: Compile and train the model
model.compile('adam', 'categorical_crossentropy', ['accuracy'])
history = model.fit(X_normalized, y_one_hot, batch_size=128, nb_epoch=10, validation_split=0.2)

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 6s - loss: 1.4843 - acc: 0.4678 - val_loss: 1.2052 - val_acc: 0.5845
Epoch 2/10
40000/40000 [==============================] - 6s - loss: 1.1170 - acc: 0.6114 - val_loss: 1.0022 - val_acc: 0.6551
Epoch 3/10
40000/40000 [==============================] - 6s - loss: 0.9597 - acc: 0.6701 - val_loss: 0.9599 - val_acc: 0.6681
Epoch 4/10
40000/40000 [==============================] - 6s - loss: 0.8420 - acc: 0.7096 - val_loss: 0.9283 - val_acc: 0.6870
Epoch 5/10
40000/40000 [==============================] - 6s - loss: 0.7338 - acc: 0.7446 - val_loss: 0.9234 - val_acc: 0.6894
Epoch 6/10
40000/40000 [==============================] - 6s - loss: 0.6242 - acc: 0.7850 - val_loss: 0.9268 - val_acc: 0.6951
Epoch 7/10
40000/40000 [==============================] - 6s - loss: 0.5210 - acc: 0.8190 - val_loss: 1.0070 - val_acc: 0.6909
Epoch 8/10
40000/40000 [==============================] - 6s 

**Best Validation Accuracy:** 69.4%

## Testing
Once you've picked out your best model, it's time to test it.

Load up the test data and use the [`evaluate()` method](https://keras.io/models/model/#evaluate) to see how well it does.

Hint 1: The `evaluate()` method should return an array of numbers. Use the [`metrics_names`](https://keras.io/models/model/) property to get the labels.

In [23]:

# TODO: Preprocess data & one-hot encode the labels
X_test, y_test = shuffle(X_test, y_test) # not really needed
min_max_range = (-0.5,0.5)
x_test_data = normalize_minmax_scale(X_test,min_max_range)
Y_test_data = lb.fit_transform(y_test)
# TODO: Evaluate model on test data
test_score = model.evaluate(x_test_data, Y_test_data)

10000/10000 [==============================] - 0s     


In [24]:
print(test_score[1])

0.6932


**Test Accuracy:** 69.3%

## Summary
Keras is a great tool to use if you want to quickly build a neural network and evaluate performance.